# Plan and Geometry Operations

# RAS Commander: Plan and Geometry Operations

This notebook demonstrates how to perform operations on HEC-RAS plan and geometry files using the RAS Commander library. We'll explore how to initialize projects, clone plans and geometries, configure parameters, execute plans, and analyze results.

## Operations Covered

1. **Project Initialization**: Initialize a HEC-RAS project by specifying the project path and version
2. **Plan Operations**:
   - Clone an existing plan to create a new one
   - Configure simulation parameters and intervals
   - Set run flags and update descriptions
3. **Geometry Operations**:
   - Clone a geometry file to create a modified version
   - Set the geometry for a plan
   - Clear geometry preprocessor files to ensure clean results
4. **Flow Operations**:
   - Clone unsteady flow files
   - Configure flow parameters
5. **Plan Computation**: Run the plan with specified settings
6. **Results Verification**: Check HDF entries to confirm results were written

## Package Installation and Environment Setup
Uncomment and run package installation commands if needed

In [ ]:
# 1. Install ras-commander from pip (uncomment to install if needed)
#!pip install ras-commander
# This installs ras-commander and all dependencies

In [ ]:
# =============================================================================# DEVELOPMENT MODE TOGGLE# =============================================================================USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS# Import Path (needed for both modes)from pathlib import Pathimport sysif USE_LOCAL_SOURCE:    local_path = str(Path.cwd().parent)    if local_path not in sys.path:        sys.path.insert(0, local_path)    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")else:    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")# Import ras-commanderfrom ras_commander import *# Additional importsimport osimport pandas as pdfrom IPython import displayfrom datetime import datetime# Verify which version loadedimport ras_commanderprint(f"✓ Loaded: {ras_commander.__file__}")

In [ ]:
# Extract specific projects we'll use in this tutorial
# This will download them if not present and extract them to the example_projects folder
bald_eagle_path = RasExamples.extract_project("Balde Eagle Creek", output_path="example_projects_02_plan_and_geometry_operations")
print(bald_eagle_path)

## Project Initialization

The first step is to initialize the HEC-RAS project. This is done using the `init_ras_project()` function, which takes the project folder path and HEC-RAS version as parameters.

In [ ]:
init_ras_project(bald_eagle_path, "6.6")
print(f"Initialized HEC-RAS project: {ras.project_name}")

# Display the current plan files in the project
print("\nHEC-RAS Project Plan Data (plan_df):")
display.display(ras.plan_df)

## Understanding Plan and Geometry Operations in HEC-RAS

Before diving into the operations, let's understand what plan and geometry files are in HEC-RAS:

- **Plan Files** (`.p*`): Define the simulation parameters including the reference to geometry and flow files, as well as computational settings.
- **Geometry Files** (`.g*`): Define the physical characteristics of the river/channel system including cross-sections, 2D areas, and structures.

The `RasPlan` and `RasGeo` classes provide methods for working with these files, including:

1. Creating new plans and geometries by cloning existing ones
2. Modifying simulation parameters and settings
3. Associating geometries with plans
4. Managing preprocessor files
5. Retrieving information from plans and geometries

In the following sections, we'll explore these operations in detail.

## Cloning Plans and Geometries

Let's start by cloning a plan to create a new simulation scenario.

In [ ]:
# Clone plan "01" to create a new plan
new_plan_number = RasPlan.clone_plan("1", new_shortid="Combined Test Plan")
print(f"New plan created: {new_plan_number}")

# Display updated plan files
print("\nUpdated plan files:")
display.display(ras.plan_df)

# Get the path to the new plan file
plan_path = RasPlan.get_plan_path(new_plan_number)
print(f"\nNew plan file path: {plan_path}")

# Let's examine the new plan's details
new_plan = ras.plan_df[ras.plan_df['plan_number'] == new_plan_number].iloc[0]
print(f"\nNew plan details:")
print(f"Plan number: {new_plan_number}")
print(f"Description: {new_plan.get('description', 'No description')}")
print(f"Short Identifier: {new_plan.get('Short Identifier', 'Not available')}")
print(f"Geometry file: {new_plan.get('Geom File', 'None')}")
print(f"File path: {new_plan['full_path']}")

In [ ]:
# Get the current plan title and shortid
current_title = RasPlan.get_plan_title(new_plan_number)
current_shortid = RasPlan.get_shortid(new_plan_number)

print(f"Current plan title: {current_title}")
print(f"Current plan shortid: {current_shortid}")

In [ ]:
# Update the title and shortid to append " clonedplan"
new_title = f"{current_title} clonedplan"
new_shortid = f"{current_shortid} clonedplan"

RasPlan.set_plan_title(new_plan_number, new_title)
RasPlan.set_shortid(new_plan_number, new_shortid)

print(f"\nUpdated plan title: {RasPlan.get_plan_title(new_plan_number)}")
print(f"Updated plan shortid: {RasPlan.get_shortid(new_plan_number)}")

In [ ]:
# Get the current plan title and shortid again to confirm the changes
current_title = RasPlan.get_plan_title(new_plan_number)
current_shortid = RasPlan.get_shortid(new_plan_number)

print(f"Current plan title: {current_title}")
print(f"Current plan shortid: {current_shortid}")

Now let's clone a geometry file. This allows us to make modifications to a geometry without affecting the original.

In [ ]:
# Clone geometry "01" to create a new geometry file
new_geom_number = RasPlan.clone_geom("01")
print(f"New geometry created: {new_geom_number}")

# Display updated geometry files
print("\nUpdated geometry files:")
display.display(ras.geom_df)

# Get the path to the new geometry file
geom_path = RasPlan.get_geom_path(new_geom_number)
print(f"\nNew geometry file path: {geom_path}")

# Examine the new geometry's details
new_geom = ras.geom_df.loc[ras.geom_df['geom_number'] == new_geom_number].squeeze()
print(f"\nNew geometry details:")
print(f"Geometry number: {new_geom_number}")
print(f"Geometry file: {new_geom.get('geom_file', 'Not available')}")
print(f"File path: {new_geom.get('full_path', 'Not available')}")
print(f"HDF path: {new_geom.get('hdf_path', 'None')}")

Let's also clone an unsteady flow file to complete our new simulation setup.

In [ ]:
# Clone unsteady flow "02" to create a new unsteady flow file
new_unsteady_number = RasPlan.clone_unsteady("02")
print(f"New unsteady flow created: {new_unsteady_number}")

# Display updated unsteady flow files
print("\nUpdated unsteady flow files:")
display.display(ras.unsteady_df)

# Examine the new unsteady flow's details
new_unsteady = ras.unsteady_df[ras.unsteady_df['unsteady_number'] == new_unsteady_number].iloc[0]
print(f"\nNew unsteady flow details:")
print(f"Unsteady number: {new_unsteady_number}")
print(f"File path: {new_unsteady['full_path']}")
print(f"Flow Title: {new_unsteady.get('Flow Title', 'Not available')}")

## Associating Files and Setting Parameters

Now that we have cloned our plan, geometry, and unsteady flow files, we need to associate them with each other and set various parameters.

### Setting Geometry for a Plan

Let's associate our new geometry with our new plan:

In [ ]:
# Set the new geometry for the cloned plan
updated_geom_df = RasPlan.set_geom(new_plan_number, new_geom_number)
plan_path = RasPlan.get_plan_path(new_plan_number, ras_object=ras)
print(f"Updated geometry for plan {new_plan_number} to geometry {new_geom_number}")
print(f"Plan file path: {plan_path}")

# Let's verify the change
updated_plan = ras.plan_df[ras.plan_df['plan_number'] == new_plan_number].iloc[0]
print(f"\nVerified that plan {new_plan_number} now uses geometry file: {updated_plan.get('Geom File', 'None')}")

### Setting Unsteady Flow for a Plan

Similarly, let's associate our new unsteady flow file with our plan:

In [ ]:
# Set unsteady flow for the cloned plan
RasPlan.set_unsteady(new_plan_number, new_unsteady_number)
print(f"Updated unsteady flow for plan {new_plan_number} to unsteady flow {new_unsteady_number}")

### Clearing Geometry Preprocessor Files

When working with geometry files, it's important to clear the preprocessor files to ensure clean results. These files (with `.c*` extension) contain computed hydraulic properties that should be recomputed when the geometry changes.

In [ ]:
# Clear geometry preprocessor files for the cloned plan
RasGeo.clear_geompre_files(plan_path)
print(f"Cleared geometry preprocessor files for plan {new_plan_number}")

# Check if preprocessor file exists after clearing
geom_preprocessor_suffix = '.c' + ''.join(Path(plan_path).suffixes[1:])
geom_preprocessor_file = Path(plan_path).with_suffix(geom_preprocessor_suffix)
print(f"Preprocessor file exists after clearing: {geom_preprocessor_file.exists()}")

### Setting Computation Parameters

Let's set the computation parameters for our plan:

In [ ]:
# Set the number of cores to use for the computation
RasPlan.set_num_cores(new_plan_number, 2)
print(f"Updated number of cores for plan {new_plan_number} to 2")

# Verify by extracting the value from the plan file
cores_value = RasPlan.get_plan_value(new_plan_number, "UNET D1 Cores")
print(f"\nVerified that UNET D1 Cores is set to: {cores_value}")

# Set geometry preprocessor options
RasPlan.set_geom_preprocessor(plan_path, run_htab=-1, use_ib_tables=-1)
print(f"Updated geometry preprocessor options for plan {new_plan_number}")
print(f"- Run HTab: -1 (Force recomputation of geometry tables)")
print(f"- Use Existing IB Tables: -1 (Force recomputation of interpolation/boundary tables)")

# Verify by extracting the values from the plan file
run_htab_value = RasPlan.get_plan_value(new_plan_number, "Run HTab")
ib_tables_value = RasPlan.get_plan_value(new_plan_number, "UNET Use Existing IB Tables")
print(f"\nVerified setting values:")
print(f"- Run HTab: {run_htab_value}")
print(f"- UNET Use Existing IB Tables: {ib_tables_value}")

### Updating Simulation Parameters

Now, let's update various simulation parameters for our plan:

In [ ]:
# 1. Update simulation date
start_date = datetime(2023, 1, 1, 0, 0)  # January 1, 2023, 00:00
end_date = datetime(2023, 1, 5, 23, 59)  # January 5, 2023, 23:59

RasPlan.update_simulation_date(new_plan_number, start_date, end_date)
print(f"Updated simulation date for plan {new_plan_number}:")
print(f"- Start Date: {start_date}")
print(f"- End Date: {end_date}")

In [ ]:
# Verify the update
sim_date = RasPlan.get_plan_value(new_plan_number, "Simulation Date")
print(f"Verified Simulation Date value: {sim_date}")

In [ ]:
# 2. Update plan intervals
RasPlan.update_plan_intervals(
    new_plan_number,
    computation_interval="1MIN",  # Computational time step
    output_interval="15MIN",      # How often results are written
    mapping_interval="30MIN"      # How often mapping outputs are created
)
print(f"\nUpdated plan intervals for plan {new_plan_number}:")
print(f"- Computation Interval: 1MIN")
print(f"- Output Interval: 15MIN")
print(f"- Mapping Interval: 30MIN")

# Verify the updates
comp_interval = RasPlan.get_plan_value(new_plan_number, "Computation Interval")
mapping_interval = RasPlan.get_plan_value(new_plan_number, "Mapping Interval")
print(f"Verified interval values:")
print(f"- Computation Interval: {comp_interval}")
print(f"- Mapping Interval: {mapping_interval}")

In [ ]:
# 3. Update run flags
RasPlan.update_run_flags(
    new_plan_number,
    geometry_preprocessor=True,   # Run the geometry preprocessor
    unsteady_flow_simulation=True, # Run unsteady flow simulation
    post_processor=True,          # Run post-processing
    floodplain_mapping=True       # Generate floodplain mapping outputs
)
print(f"\nUpdated run flags for plan {new_plan_number}:")
print(f"- Geometry Preprocessor: True")
print(f"- Unsteady Flow Simulation: True")
print(f"- Post Processor: True")
print(f"- Floodplain Mapping: True")

In [ ]:
# Verify the updates
run_htab = RasPlan.get_plan_value(new_plan_number, "Run HTab")
run_unet = RasPlan.get_plan_value(new_plan_number, "Run UNet")
print(f"Verified run flag values:")
print(f"- Run HTab (Geometry Preprocessor): {run_htab}")
print(f"- Run UNet (Unsteady Flow): {run_unet}")

# 4. Update plan description
new_description = "Combined plan with modified geometry and unsteady flow\nJanuary 2023 simulation\n1-minute computation interval\nGeometry and unsteady flow from cloned files"
RasPlan.update_plan_description(new_plan_number, new_description)
print(f"\nUpdated description for plan {new_plan_number}")

# Read back the description
current_description = RasPlan.read_plan_description(new_plan_number)
print(f"Current plan description:\n{current_description}")

## Computing the Plan

Now that we have set up all the parameters, let's compute the plan using RasCmdr.compute_plan():

In [ ]:
# Compute the plan with our configured settings
# Note: This may take several minutes depending on the complexity of the model
print(f"Computing plan {new_plan_number}...")
success = RasCmdr.compute_plan(new_plan_number, clear_geompre=True)

if success:
    print(f"Plan {new_plan_number} computed successfully")
else:
    print(f"Failed to compute plan {new_plan_number}")

## Verifying Results

After computation, we should check if results were written correctly:

In [ ]:
# Refresh the plan entries to ensure we have the latest data
ras.plan_df = ras.get_plan_entries()
hdf_entries = ras.get_hdf_entries()

if not hdf_entries.empty:
    print("HDF entries for the project:")
    display.display(hdf_entries)
    
    # Check if our new plan has an HDF file
    new_plan_hdf = hdf_entries[hdf_entries['plan_number'] == new_plan_number]
    if not new_plan_hdf.empty:
        print(f"\nPlan {new_plan_number} has a valid HDF results file:")
        print(f"HDF Path: {new_plan_hdf.iloc[0]['HDF_Results_Path']}")
    else:
        print(f"\nNo HDF entry found for plan {new_plan_number}")
else:
    print("No HDF entries found. This could mean the plan hasn't been computed successfully or the results haven't been written yet.")

# Display all plan entries to see their HDF paths
print("\nAll plan entries with their HDF paths:")
plan_hdf_info = ras.plan_df[['plan_number', 'HDF_Results_Path']]
display.display(plan_hdf_info)

If the plan was computed successfully, we can examine the runtime data and volume accounting from the HDF results:

In [ ]:
# Get computation runtime data from HDF
print("Checking computation runtime data...")
runtime_df = HdfResultsPlan.get_runtime_data(new_plan_number)

if runtime_df is not None and not runtime_df.empty:
    print("\nSimulation Runtime Statistics:")
    display.display(runtime_df)
    
    # Extract key metrics
    sim_duration = runtime_df['Simulation Duration (s)'].iloc[0]
    compute_time = runtime_df['Complete Process (hr)'].iloc[0]
    compute_speed = runtime_df['Complete Process Speed (hr/hr)'].iloc[0]
    
    print(f"\nSimulation Duration: {sim_duration:.2f} seconds")
    print(f"Computation Time: {compute_time:.5f} hours")
    print(f"Computation Speed: {compute_speed:.2f} (simulation hours/compute hours)")
else:
    print("No runtime data found. This may indicate the simulation didn't complete successfully.")

# Get volume accounting data
print("\nChecking volume accounting...")
volume_df = HdfResultsPlan.get_volume_accounting(new_plan_number)

if volume_df is not None and not isinstance(volume_df, bool):
    # Handle volume_df as a dictionary
    if isinstance(volume_df, dict):
        error_percent = volume_df.get('Error Percent')
        if error_percent is not None:
            print(f"\nFinal Volume Balance Error: {float(error_percent):.8f}%")
            
        # Print other key statistics
        print("\nDetailed Volume Statistics:")
        print(f"Volume Starting: {float(volume_df['Volume Starting']):.2f} {volume_df['Vol Accounting in'].decode()}")
        print(f"Volume Ending: {float(volume_df['Volume Ending']):.2f} {volume_df['Vol Accounting in'].decode()}")
        print(f"Total Inflow: {float(volume_df['Total Boundary Flux of Water In']):.2f} {volume_df['Vol Accounting in'].decode()}")
        print(f"Total Outflow: {float(volume_df['Total Boundary Flux of Water Out']):.2f} {volume_df['Vol Accounting in'].decode()}")
else:
    print("No volume accounting data found. This may indicate the simulation didn't complete successfully.")

## Summary of Plan and Geometry Operations

In this notebook, we've covered a comprehensive range of operations on HEC-RAS plan and geometry files using the RAS Commander library:

1. **Project Initialization**: We initialized a HEC-RAS project to work with
2. **Plan Operations**:
   - Created a new plan by cloning an existing one
   - Updated simulation parameters (dates, intervals, etc.)
   - Set run flags for different components
   - Updated the plan description
3. **Geometry Operations**:
   - Created a new geometry by cloning an existing one
   - Associated the new geometry with our plan
   - Cleared geometry preprocessor files
4. **Unsteady Flow Operations**:
   - Created a new unsteady flow file by cloning an existing one
   - Associated it with our plan
5. **Computation and Verification**:
   - Computed our plan with the specified settings
   - Verified the results using HDF entries
   - Analyzed runtime statistics and volume accounting


### Key Classes and Functions Used

- `RasPlan`: For plan operations (cloning, setting components, and modifying parameters)
- `RasGeo`: For geometry operations (cloning, clearing preprocessor files)
- `RasCmdr`: For executing HEC-RAS simulations

### Next Steps

To further enhance your HEC-RAS automation, consider exploring:

1. **Parameter Sweeps**: Create and run multiple plans with varying parameters
2. **Parallel Computations**: Run multiple plans simultaneously using `RasCmdr.compute_parallel()`
3. **Advanced Results Analysis**: Use the HDF classes to extract and analyze specific model results
4. **Spatial Visualization**: Create maps and plots of simulation results
5. **Model Calibration**: Automate comparison between model results and observations

The RAS Commander library provides a powerful framework for automating and streamlining your HEC-RAS workflows, enabling more efficient hydraulic modeling and analyses.